# TextRank for Document Summarization

In [19]:
import pandas as pd
import numpy as np

from konlpy.tag import Mecab
from nltk import sent_tokenize
import kss

In [2]:
text = "딸기 바나나 사과 파인애플 수박. 바나나 사과 딸기 포도. 복숭아 수박. 파인애플 사과 딸기 바나나."

## 문장 유사도 계산

In [9]:
mecab = Mecab()
def sentence_sim(s1, s2):
    s1_tokens = {token for token in mecab.pos(s1)
                    if token[1][0] in ['N', 'V']}
    s2_tokens = {token for token in mecab.pos(s2)
                    if token[1][0] in ['N', 'V']}

    union = s1_tokens.union(s2_tokens)
    inter = s1_tokens.intersection(s2_tokens)    
    return len(inter)/len(union)

In [12]:
sentence_sim('나는 치킨 좋아라 해', '나는 피자 좋아 해')

0.6

In [14]:
text_split = text.split('.')[:-1]
text_split

['딸기 바나나 사과 파인애플 수박', ' 바나나 사과 딸기 포도', ' 복숭아 수박', ' 파인애플 사과 딸기 바나나']

In [21]:
kss.split_sentences(text)

['딸기 바나나 사과 파인애플 수박. 바나나 사과 딸기 포도. 복숭아 수박. 파인애플 사과 딸기 바나나.']

In [22]:
sent_tokenize(text)

['딸기 바나나 사과 파인애플 수박.', '바나나 사과 딸기 포도.', '복숭아 수박.', '파인애플 사과 딸기 바나나.']

## buildMatrix 함수

In [23]:
def buildMatrix(sentences):
    leng = len(sentences)
    score = np.ones(leng, dtype=np.float32)
    weighted_edge = np.zeros((leng, leng), dtype=np.float32)
    
    for i in range(leng):
        for j in range(leng):
            if i==j:
                # 자기 자신은 0이므로 계산하지 않고 넘어감
                continue
            weighted_edge[i][j] = sentence_sim(sentences[i],
                                              sentences[j])
    #return weighted_edge
    for i in range(len(weighted_edge)):
        score[i] = weighted_edge[i].sum()
        weighted_edge[i] /= score[i]
        
    return score, weighted_edge

In [16]:
buildMatrix(text_split)

array([[0.        , 0.5       , 0.16666667, 0.8       ],
       [0.5       , 0.        , 0.        , 0.6       ],
       [0.16666667, 0.        , 0.        , 0.        ],
       [0.8       , 0.6       , 0.        , 0.        ]], dtype=float32)

In [24]:
buildMatrix(text_split)

(array([1.4666667 , 1.1       , 0.16666667, 1.4000001 ], dtype=float32),
 array([[0.        , 0.3409091 , 0.11363637, 0.54545456],
        [0.45454544, 0.        , 0.        , 0.54545456],
        [1.        , 0.        , 0.        , 0.        ],
        [0.57142854, 0.4285714 , 0.        , 0.        ]], dtype=float32))

## Scoring

In [25]:
def scoring(W, S, eps = 0.0001, d=0.85, max_iter=50):
    
    for iter_ in range(max_iter):
        new_S = (1 - d) + d * np.dot(W.T, S)
        
        if (abs(new_S - S) < eps).all():
            return new_S
        
        S = new_S
    
    return new_S

In [94]:
def summarize(text, n=10):
    sentences = list(set(sent_tokenize(text)))
    score, weighted_edge = buildMatrix(sentences)
    score = scoring(weighted_edge, score)
    
    return [(sentences[idx], score)
                  for idx, score in sorted(enumerate(score),
                  key=lambda x: x[1],
                  reverse=True)]

In [95]:
summarize(text)

[('딸기 바나나 사과 파인애플 수박.', 1.4108208),
 ('파인애플 사과 딸기 바나나.', 1.2793751),
 ('바나나 사과 딸기 포도.', 1.0249318),
 ('복숭아 수박.', 0.286281)]

## Graph Tfidf

In [40]:
import networkx as nx

In [75]:
def rank(nodes, edges):
    graph = nx.diamond_graph()
    graph.clear()
    graph.add_nodes_from(nodes)
    graph.add_weighted_edges_from(edges)
    return nx.pagerank(graph)

In [76]:
def connect(nodes):
    return [(start, end, sentence_sim(start, end)) for start in nodes for end in nodes
            if start != end]

In [87]:
def summarize_nx(text, n=10):
    nodes = set(sent_tokenize(text))
    edges = connect(nodes)
    scores = rank(nodes, edges)
    return sorted(scores.items(), key=lambda x: x[1],
                 reverse=True)[:n]

In [88]:
summarize_nx(text, 3)

[('딸기 바나나 사과 파인애플 수박.', 0.35258089874833043),
 ('파인애플 사과 딸기 바나나.', 0.31972392073355704),
 ('바나나 사과 딸기 포도.', 0.2561392427214696)]

## 중복된 문장?


In [89]:
text2 = "딸기 바나나 사과 파인애플 수박. 바나나 사과 딸기 포도. 복숭아 수박. 복숭아 수박. 파인애플 사과 딸기 바나나. 파인애플 사과 딸기 바나나."

In [96]:
summarize(text2)

[('딸기 바나나 사과 파인애플 수박.', 1.4108208),
 ('파인애플 사과 딸기 바나나.', 1.2793751),
 ('바나나 사과 딸기 포도.', 1.0249318),
 ('복숭아 수박.', 0.286281)]

In [91]:
summarize_nx(text2)

[('딸기 바나나 사과 파인애플 수박.', 0.35258089874833043),
 ('파인애플 사과 딸기 바나나.', 0.31972392073355704),
 ('바나나 사과 딸기 포도.', 0.2561392427214696),
 ('복숭아 수박.', 0.07155593779664282)]

## gensim.summarization.summarizer

In [98]:
from gensim.summarization.summarizer import summarize

In [102]:
summarize(text)

''